1.먼저 F 빼고 다 지웁시다.

In [65]:
import json
import os
import math
import pandas as pd
import cv2

- data 디렉토리는 설정해주세요. 저같은 경우는 ../data/에 있습니다.

In [2]:
file_dir = "../data/" # setting

- dir 세팅

In [3]:
step1 = os.listdir(file_dir)
step1

['CROWD_REAL005', 'CROWD_REAL006', 'CROWD_REAL004']

- 차례차례 들어가볼게요

In [86]:
for dir1 in step1:
    step1_dir = file_dir + dir1
    step2 = os.listdir(step1_dir)
#     print(step1_dir)
    
    for dir2 in step2:
        step2_dir = step1_dir + '/' + dir2
        step3 = os.listdir(step2_dir)
        
        for dir3 in step3:
            step3_dir = step2_dir + '/' + dir3
            file_list = os.listdir(step3_dir)
            
            ## 여기 file list 파싱해서 F 다 지웁시다.
            for file in file_list:
                file_name = step3_dir + '/' + file
                if file.split('.')[0].split('_')[4] != 'F': # 삭제
                    os.remove(file_name)
#                     print(file_name) # 지워진 파일 출력

2. 영상 잘라서 이미지 만들기

- 데이터가 비어있는 영상도 체크합시다. 
- 저희는 [영상제목, 영상 시작시간, 영상 종료시간] 정보로 이미지 만들게요.

In [54]:
video_start_end_info = []
error_video = []

In [55]:
for dir1 in step1:
    step1_dir = file_dir + dir1
    step2 = os.listdir(step1_dir)
#     print(step1_dir)
    
    for dir2 in step2:
        step2_dir = step1_dir + '/' + dir2
        step3 = os.listdir(step2_dir)
        
        # 형태소로 들어가기 및 영상 저장
        video_dir = step2_dir + '/' + step3[0]
        step3_dir = step2_dir + '/' + step3[2]
        file_list = os.listdir(step3_dir)
        
        print("파일 갯수", len(file_list))
        
        for file in file_list:
            tmp = []
            
            file_name = step3_dir + '/' + file
#             print(file_name)
            
            with open(file_name) as json_file:
                try:
                    json_data = json.load(json_file)
                    clip_name = json_data["metaData"]['name']
    #                 print(clip_name)
    
                    if len(json_data["data"]) != 0:
                        clip_start = json_data["data"][0]['start']
                        tmp.append(video_dir + '/' + clip_name)
                    else:
                        print("에러난 파일명: ", clip_name)
                        error_video.append(file_name)
                        continue
                    
                    clip_start = math.trunc(clip_start * 10) / 10
                    tmp.append(clip_start)
                #     print(clip_start)

                    clip_end = json_data["data"][0]['end']
                    clip_end = math.ceil(clip_end * 10) / 10
                    tmp.append(clip_end)
                #     print(clip_end)
                except:
                    print("에러난 파일명: ", clip_name) # 아마 data가 없는파일임.
            video_start_end_info.append(tmp)

파일 갯수 1000
에러난 파일명:  NIA_SL_FS0713_CROWD12_F.mp4
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000
에러난 파일명:  NIA_SL_FS0795_CROWD14_F.MP4
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000


- 갯수 체크하시구

In [56]:
print("정상적으로 담긴거 ", len(video_start_end_info))
print("에러 영상 ", len(error_video))

정상적으로 담긴거  8998
에러 영상  2


- 잘 담겼는지 확인해주세요~

In [60]:
pd_test = pd.DataFrame(video_start_end_info)
pd_test.head()

,0,1,2
0,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.5,6.4
1,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.0,7.8
2,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.5,5.4
3,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.9,6.7
4,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.1,5.4


- 이미지를 저장할 폴더를 만들고 잘라봅시다.

In [63]:
img_dir = "../data/img"
os.makedirs(img_dir)

- 시작합니다~ 추가로 한번 진행해보다가 중단하시고 프레임번호가 위에 영상 시작, 끝시간과 맞는지 확인해보는거도 조아요~

In [ ]:
for video_info in video_start_end_info:
    # video_info[0] : 영상 경로, video_info[1] : 영상 시작, video_info[2] : 영상 끝
    
    editing_video = video_info[0].split('/')[-1]
    output_dir = img_dir + '/' + editing_video
    os.makedirs(output_dir)
    
    vidcap = cv2.VideoCapture(video_info[0])
    count = -1
    while vidcap.isOpened():
        count+=1
        success, image = vidcap.read()
        if success: 
            if video_info[1] * 30 <= count and count <= video_info[2] * 30:
                cv2.imwrite(os.path.join(output_dir, editing_video+ ' %d.png') %(count), image)
        else:
            cv2.destroyAllWindows()
            vidcap.release()